In [3]:
import pandas as pd
import sys
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from collections import Counter

#preprocess all_agree_filtered data
all_agreed_data_file = "/efs/lhoang2/all_agree_filtered.txt"
df = pd.read_csv(all_agreed_data_file, sep = "|",header = None)
df.columns = ["PMCID","sentence_id","CONSORT_Item","sentence"]
df.head(10)

PMCID_list = df["PMCID"]
unique_PMCID_list = list(set(PMCID_list))
print ("Number of unique PMCIDs: ", len(unique_PMCID_list))

sentence_id_list = df["sentence_id"]
print ("Number of sentences: ", len(sentence_id_list))

labels = df["CONSORT_Item"]
labels = labels.tolist()
value_counts = Counter(labels)
print (value_counts)


#drop CONSORT items NO_AGR, 8a, 12b, 9
array = ['NO_AGR', '8a', '12b', '9']
filtered_df = df.loc[~df['CONSORT_Item'].isin(array)]

Number of unique PMCIDs:  17357
Number of sentences:  993984
Counter({'NO_AGR': 555589, 'NONE': 400055, '12a': 25821, '4a': 4715, '7a': 3139, '6a': 1854, '3a': 1451, '5': 657, '11a': 300, '10': 129, '8b': 127, '4b': 112, '7b': 28, '9': 3, '12b': 3, '8a': 1})


In [ ]:
### PREPROCESSING AUTOMATIC ANNOTATED DATA ####

from xml.etree import ElementTree as ET
import os
import pandas as pd

def read_xml_section(filePath):
    PMCID = filePath.replace(".xml","")
    PMCID = PMCID.replace("/efs/CONSORT/PMC_Internal/","")
    tree = ET.parse(filePath)
    root = tree.getroot()
    
    document_id_list = []
    section_title_list = []
    section_start_list = []
    section_end_list = []
    section_sub_or_not_list = []
    
    for child in root:
        if child.tag == "section":
            if len(child) == 0:
                sectionTextSpan = child.attrib["textSpan"]                
                if 'title' in child.attrib:
                    sectionTitle = child.attrib["title"]
                else:
                    sectionTitle = "None"
                
                sectionSubSectionOrNot = "No"
                startSectionChar, endSectionchar = sectionTextSpan.split("-")
                     
                document_id_list.append(PMCID)
                section_title_list.append(sectionTitle)
                section_start_list.append(startSectionChar)
                section_end_list.append(endSectionchar)
                section_sub_or_not_list.append(sectionSubSectionOrNot)
                    
            if len(child) != 0:
                sectionTextSpan = child.attrib["textSpan"]
                if 'title' in child.attrib:
                    sectionTitle = child.attrib["title"]
                else:
                    sectionTitle = "None"
                startSectionChar, endSectionchar = sectionTextSpan.split("-")
                sectionSubSectionOrNot = "No"
                
                document_id_list.append(PMCID)
                section_title_list.append(sectionTitle)
                section_start_list.append(startSectionChar)
                section_end_list.append(endSectionchar)
                section_sub_or_not_list.append(sectionSubSectionOrNot)
                
                
                for child_level1 in child:
                    subSectionTextSpan = child_level1.attrib["textSpan"]
                    if 'title' in child_level1.attrib:
                        subSectionTitle = child_level1.attrib["title"]
                    else:
                        subSectionTitle = "None"
                    startSubSectionChar, endSubSectionchar = subSectionTextSpan.split("-")
                    sectionSubSectionOrNot = "Yes"
                                        
                    document_id_list.append(PMCID)
                    section_title_list.append(subSectionTitle)
                    section_start_list.append(startSubSectionChar)
                    section_end_list.append(endSubSectionchar)
                    section_sub_or_not_list.append(sectionSubSectionOrNot)
    
    section_df = pd.DataFrame(
    {'PMCID': document_id_list,
     'section_title': section_title_list,
     'section_start': section_start_list,
     'section_end': section_end_list,
     'subsection?': section_sub_or_not_list
    })
    return section_df

                    
def read_xml(filePath):
    section_df = read_xml_section(filePath)
    
    PMCID = filePath.replace(".xml","")
    PMCID = PMCID.replace("/efs/CONSORT/PMC_Internal/","")
    tree = ET.parse(filePath)
    root = tree.getroot()
    
    document_id_list = []
    sentence_list = []
    sentence_id_list = []
    start_char_list = []
    end_char_list = []
    section_list = []

    for child in root:
        if child.tag == "sentence":
            charOffSet = (child.attrib["charOffset"])
            sentence_id= (child.attrib["id"])
            start_char,end_char = charOffSet.split("-")
            
            for index_section,row_section in section_df.iterrows():
                if (PMCID == row_section["PMCID"] and int(start_char)>=int(row_section["section_start"]) and int(end_char)<=int(row_section["section_end"])):
                    section = row_section["section_title"]
            
            for child_level1 in child:
                if child_level1.tag == "text":
                    sentence_text = child_level1.text
                    
                    document_id_list.append(PMCID)
                    sentence_id_list.append(sentence_id)
                    sentence_list.append(sentence_text)
                    start_char_list.append(start_char)
                    end_char_list.append(end_char)
                    section_list.append(section)

    df = pd.DataFrame(
    {'PMCID': document_id_list,
     'sentence_id': sentence_id_list,
     'sentence_text': sentence_list,
     'start_char_pos': start_char_list,
     'end_char_pos': end_char_list,
     'section': section_list
    })
    return df

data_path = "/efs/CONSORT/PMC_Internal/"
filelist = os.listdir(data_path)

columns=['PMCID', 'sentence_id','sentence_text', 'start_char_pos','end_char_pos','section']
all_data_df = pd.DataFrame(columns=columns)

count = 0
for i in filelist:
    if i.endswith("xml"):
        file_path = data_path + i
        print (file_path)
        count +=1
        print (count)
        data_df = read_xml(file_path)
        all_data_df = pd.concat([all_data_df,data_df])

In [ ]:
merged_df = pd.merge(filtered_df, all_data_df, on=["PMCID","sentence_id"], how="right")
merged_df.to_csv("/efs/lhoang2/All_Automatic_Annotated_Data_(TrainingSet).csv",index=False)